# Cell Lysis modelling

Code implemented by Mariana Albino, marial@kt.dtu.dk

In [1]:
# import libraries
import numpy as np
import scipy
import plotly.express as px
import plotly.graph_objects as go
from scipy.integrate import solve_ivp
from plotly.subplots import make_subplots

ModuleNotFoundError: No module named 'numpy'

## Bead mill

In [2]:
def ode_fcn(t, y, k, Rm):
    # define parameters
    R = y[0]

    # ODE
    dR_dt = k*(Rm-R)
    dy = [dR_dt]

    return dy

def solver(R_0, k ,Rm ,t_end):
    #define the function to solve
    fun = lambda t, y: ode_fcn(t,y,k,Rm)

    #set initial value for parameters
    y0 = [R_0]

    #set time span
    t_span = np.arange(0, t_end, 0.1)  #(t_start, t_end, t_step)

    #solve the differential equation
    sol = solve_ivp(fun, [t_span[0], t_span[-1]], y0, method='LSODA', t_eval=t_span, rtol=1e-6, atol=1e-6)

    #extract the solution
    t = sol.t.tolist()
    y = sol.y.T

    #asign the solution to variables
    R = y[:, 0]
    Y = y[:,0]/Rm*100

    return t, R, Y

In [3]:
""" Set Process Parameters for run generation """
R_0 = 0 #R at time 0
R_m = 10 #Rm
K = 0.5

TIME_END = 5 #how much time should be simulated

t, R , Y = solver(R_0, K, R_m, TIME_END) #run the solver function

NameError: name 'np' is not defined

In [4]:
#create the result plots
fig = px.line(x=t, y=R) #plot time vs. R
fig2=px.line(x=t, y=Y) #plot time vs. Y

#Define the settings for the figures
fig.update_layout(title="Evolution of R",xaxis_title="t",yaxis_title="R",width=1000)
fig2.update_layout(title="Evolution of Yield",xaxis_title="t",yaxis_title="Y (%)", width=1000)

#Display the figures
fig.show()
fig2.show()

NameError: name 'px' is not defined

## **High pressure homogeniser**

In [5]:
def ode_fcn_hph(n, y, k, Rm, P, a):
    # define parameters
    R = y[0]

    # mass balances
    dR_dt = k*(P**a)*(Rm-R)
    dy = [dR_dt]
    return dy

def solver_hph(R_0, k ,Rm , P, a ,N):
    fun = lambda n, y: ode_fcn_hph(n,y,k,Rm, P, a)
    y0 = [R_0]
    N_span = np.arange(0, N+1, 1)
    sol = solve_ivp(fun, [N_span[0], N_span[-1]], y0, method='LSODA', t_eval=N_span, rtol=1e-6, atol=1e-6)
    n = sol.t.tolist()
    y = sol.y.T
    R = y[:, 0]
    Y = y[:,0]/Rm*100

    return n, R, Y


In [6]:
""" Set Process Parameters for run generation """
R_0_hph = 0
R_m_hph = 100
K_hph = 5.9e-4
a = 1.77   #alpha=0.9-2.9 (2.2 E. coli; 2.9 S. cerevisiae)
N=10

#range of P to test
P = [50, 75, 100, 125]


#list to store the results of the iteration
all_results = []
#open two figures
fig_hph = make_subplots()
fig2_hph = make_subplots()

#iterate the solver function for all p values
for p in P:
    n, R_hph, Y_hph = solver_hph(R_0_hph, K_hph, R_m_hph, p, a, N)
    all_results.append((n, R_hph, Y_hph))

    #add result for current P to the figures
    fig_hph.add_trace(go.Scatter(x=n,y=R_hph, mode='markers+lines',name=p))
    fig2_hph.add_trace(go.Scatter(x=n,y=Y_hph, mode='markers+lines',name=p))

#Define the settings for the figures
fig_hph.update_layout(showlegend=True,legend_title='Pressure', title="Evolution of R",xaxis_title="N",yaxis_title="R",width=1000)
fig2_hph.update_layout(showlegend=True,legend_title='Pressure', title="Evolution of Yield",xaxis_title="N",yaxis_title="Y (%)",width=1000)

#Display the figures
fig_hph.show()
fig2_hph.show()



NameError: name 'make_subplots' is not defined